In [1]:
import sys
import os
from os.path import join as pj
from copy import deepcopy
from astropy.io import fits
# from os.path import exists

_HOME_DIR = os.path.expanduser("~")    

try:
    _SPARCFIRE_DIR = os.environ["SPARCFIRE_HOME"]
    _MODULE_DIR = pj(_SPARCFIRE_DIR, "GalaxyMusic", "GalfitModule")
except KeyError:
    # print("SPARCFIRE_HOME is not set. Please run 'setup.bash' inside SpArcFiRe directory if not done so already.")
    # print("Running on the assumption that GalfitModule is in your home directory... (if not this will fail and quit!)") 
    _MODULE_DIR = pj(_HOME_DIR, "GalaxyMusic", "GalfitModule")
    
sys.path.append(_MODULE_DIR)
    
from Classes.Components import *
from Classes.Containers import *
from Functions.helper_functions import *
from sparc_to_galfit_feedme_gen import *

In [2]:
#Imports
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
import numpy as np
import ipywidgets as widgets
from IPython.display import display

In [3]:
#Imports
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
import numpy as np
import ipywidgets as widgets
from IPython.display import display

In [4]:
import sounddevice as sd
import numpy as np
from scipy import fft
import matplotlib.pyplot as plt

In [5]:
def music_to_galaxy_properties(dict_music):
    '''
    Bb – ‘Magnitude’ [3,25] -> Linearly [0,1] | Set: 15 | Direction: Longer -> Lower Value
    F – ‘Eff_Rad’ [0,1] | Set: 0.5 | Direction: Longer -> Higher Value
    C – ‘Sersic_Index’ [0,10] (log scaling) | Set: 1.0 | Direction: Longer -> Higher Value
    G – ‘Axis_Ratio’ [0.5,1] | Set: 1.0 | Direction: Longer -> Lower Value
    D – ‘Pos_Angle’ [0,180] -> [0,1] Sin(theta/2) | Set: 0 | Direction: Longer -> Higher Value
    
    Returns: 
        disk: Float, 
        bulge: Float,
        dict_galfit: Dictionary
    '''

    dict_galfit = {}

    if dict_music['M'] == 1:
        disk = 1.0
        bulge = 0.0

    elif dict_music['m'] == 1:
        disk = 0.0
        bulge = 1.0

    def normalize(val, val_max=5.0, val_min=1.0):
        '''
        Normalizes input duration to [0,1]
        '''
        norm = (val - val_min)/(val_max - val_min)
        return norm

    dict_galfit['magnitude'] = (25.-3.)*(1.0 - normalize(dict_music['Bb']))+3.
    dict_galfit['effective_radius'] = (1.0 - 0.0)*normalize(dict_music['F'])+ 0.0
    dict_galfit['sersic_index'] = (10.0 - 0.0)*normalize(dict_music['C']) + 0.0
    dict_galfit['axis_ratio'] = (1 - 0.5)*normalize(dict_music['G']) + 0.5
    dict_galfit['position_angle'] = 2*np.arcsin(normalize(dict_music['D']))*180.0/np.pi
    
    return (disk, bulge, dict_galfit) 

In [6]:
g_prefix = "galaxy"
g_number = 0

IN_DIR  = pj(_HOME_DIR, "GalaxyMusic", "music-in")
TMP_DIR = pj(_HOME_DIR, "GalaxyMusic", "music-tmp")
OUT_DIR = pj(_HOME_DIR, "GalaxyMusic", "music-out")

run_galfit = pj(_MODULE_DIR, "galfit")

previous = False

duration = 10
fs = 48000
Nsamples = int(duration * fs)

count = 0

In [7]:
# Function to update the FITS image when slider value changes
def update_image_fits(change):
    # Update the image data based on the slider value
    new_image_data_fits = image_data_fits * change.new

    # Update the FITS image with new data
    image_fits.set_data(new_image_data_fits)

    # Redraw the figure
    fig.canvas.draw()

# Function to update the array image when slider value changes
def update_image_array(change):
    # Update the image data based on the slider value
    new_image_data_array = image_data_array * change.new

    # Update the array image with new data
    image_array.set_data(new_image_data_array)

    # Redraw the figure
    fig.canvas.draw()

In [9]:
while g_number == 0:
    gname = f"{g_prefix}_{g_number}"
    filename = f"{pj(IN_DIR, gname)}.fits"
    template = pj(_MODULE_DIR, "template.fits")
    sp(f"cp {template} {filename}")
    #gen_template(filename)

    if not exists(pj(OUT_DIR, gname)):
        os.mkdir(pj(OUT_DIR, gname))

    feedme = pj(OUT_DIR, gname, f"{gname}.in")
    output_filename = pj(OUT_DIR, gname, f"{gname}.fits")
    
    x = 10000
    
    while x != 0 and count < 100:
        x = int(input("Waiting..."))
        fs = 48000
        duration = 6
        Nsamples = int(duration*fs)
        Cmaj = sd.rec(Nsamples, samplerate=fs, channels=1, blocking=True)[:,0]#dur5
        r = Cmaj
        t = np.arange(Nsamples)/fs
        #extracting chord and duration
        #ignore first 0.5 seconds
        # blur(abs(signal)) to find peaks
        # FT first peak to get chord
        # output as string
        # time between this and last peak, output this too

        signal = Cmaj

        cutsignal = signal[t>0.5]
        cutt = t[t>0.5]
        s = 0.01
        blurProfile = (1/np.sqrt(2*np.pi*s**2))*np.exp(-0.5*(np.linspace(-3*s, 3*s, int(6*s*fs))/s)**2)

        signalBlurred = np.convolve(cutsignal**2, blurProfile, mode='same')

        chordrange = signalBlurred>100

        FT = fft.fft(cutsignal[chordrange])
        freqs = fft.fftfreq(len(cutsignal[chordrange]), d=1/fs)

        FT = FT[freqs>=0]
        freqs = freqs[freqs>=0]

        fmask = (freqs>100)&(freqs<2000)

        noteBins = 220*(2**((np.arange(12*3)-0.5)/12)) # starting from A below middle

        peaks = np.argsort(FT)[-3:]
        
        sectionMask = (t>0.6)&(t<0.65)

        section = r[sectionMask]

        FT = fft.fft(section)
        freqs = fft.fftfreq(len(section), d=1/fs)

        mask = (freqs>10)&(freqs<2500)
        
        if previous:
            galaxy_feedme = previous
        else:
            galaxy_feedme = FeedmeContainer(path_to_feedme = feedme)

            galaxy_feedme.header.param_values.update({
                "input_image"  : filename,
                "output_image" : output_filename,
                "optimize"     : 1,
                "region_to_fit" : (0,148,0,148)
                }
            )

        disk, bulge, dict_galfit = music_to_galaxy_properties(dict_music)
        dict_galfit.update({"position" : (74, 74)})

        if bulge == 1.0:
            galaxy_feedme.bulge.param_values.update(dict_galfit)
        elif disk == 1.0:
            galaxy_feedme.disk.param_values.update(dict_galfit)
        else:
            print("something went wrong")

        galaxy_feedme.update_components(bulge = bulge, disk = disk)

        #print(galaxy_feedme)
        process = sp(f"{run_galfit} {feedme}")

        previous = deepcopy(galaxy_feedme)
        #print(process.stdout)
    
        # Open the FITS file
        fits_file = fits.open(output_filename)

        # Extract the image data from the FITS file
        image_data_fits = fits_file[0].data

        # Close the FITS file
        fits_file.close()
        # Create a figure and subplots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

        # Display the FITS image
        image_fits = ax1.imshow(image_data_fits, cmap='gray')

        # Display the array image
        image_array = ax2.imshow(image_data_array, cmap='gray')

        # Create a slider widget for the FITS image
        slider_fits = widgets.FloatSlider(value=1.0, min=0.1, max=2.0, step=0.1, description='FITS Scale:', continuous_update=True)

        # Create a slider widget for the array image
        slider_array = widgets.FloatSlider(value=1.0, min=0.1, max=2.0, step=0.1, description='Array Scale:', continuous_update=True)

        # Attach the update_image_fits function to the FITS slider's value change event
        slider_fits.observe(update_image_fits, 'value')

        # Attach the update_image_array function to the array slider's value change event
        slider_array.observe(update_image_array, 'value')

        # Display the slider widgets
        display(widgets.HBox([slider_fits, slider_array]))

        # Add any additional customization you need to the subplots
        ax1.set_title('FITS Image')
        ax1.set_xlabel('X-axis')
        ax1.set_ylabel('Y-axis')

        ax2.set_title('Residual')
        ax2.set_xlabel('X-axis')
        ax2.set_ylabel('Y-axis')

        # Show the plot
        plt.show()
            
        x = input()
        count += 1
        
    g_number += 1

Waiting...1
||PaMacCore (AUHAL)|| AUHAL component not found.

PortAudioError: Error querying host API -9979

In [ ]:
#!pip install --force-reinstall 'cffi>=1.15.1'